In [1]:
%run library.ipynb

In [75]:
G = nr.binomial(2,0.5,(1000,300)).astype('float')

In [3]:
def cor_jackknife(X):
    X2  = X**2
    X12 = X.prod(axis=1)

    Xsum   = X.sum(axis=0)
    X2sum  = X2.sum(axis=0)
    X12sum = X12.sum()
    
    Xisum   = Xsum[None, :] - X
    X2isum  = X2sum[None, :] - X2
    X12isum = X12sum - X12
    
    N = len(X12)
    
    r = (N*X12sum - Xsum.prod()) / np.sqrt((N*X2sum - Xsum**2).prod())
    ris = ((N-1)*X12isum - Xisum.prod(axis=1)) / np.sqrt(((N-1)*X2isum - Xisum**2).prod(axis=1))
    rimean = ris.mean()
    ri2mean = (ris**2).mean()
    
    return(r, r**2, rimean, ri2mean, N*r - (N-1)*rimean, N*(r**2) - (N-1)*ri2mean)

In [9]:
def cor_jackknife2(A, B):
    A2 = A**2
    B2 = B**2
    AB = A * B[:,None]

    Asum = A.sum(axis=0)
    Bsum = B.sum()

    A2sum = A2.sum(axis=0)
    B2sum = B2.sum()

    ABsum = AB.sum(axis=0)

    Aisum   = Asum[None, :] - A
    Bisum   = Bsum - B

    A2isum  = A2sum[None, :] - A2
    B2isum  = B2sum - B2

    ABisum = ABsum[None, :] - AB

    N = len(B)

    r = (N*ABsum - Asum * Bsum) / np.sqrt((N*A2sum - Asum**2) * (N*B2sum - Bsum**2))
    ris = ((N-1)*ABisum - Aisum*Bisum[:,None]) / np.sqrt(((N-1)*A2isum - Aisum**2) * ((N-1)*B2isum - Bisum**2)[:,None])
    rimean = ris.mean(axis=0)
    ri2mean = (ris**2).mean(axis=0)

    return(r, r**2, rimean, ri2mean, N*r - (N-1)*rimean, N*(r**2) - (N-1)*ri2mean)

In [94]:
M = G.shape[1]
N = int(M*(M+1)/2)

In [96]:
#%%timeit

qs = np.ones(N)
q2s = np.ones(N)

k = 0
for i in range(M):
    a, b, c, d, r, r2 = cor_jackknife2(G[:,:i], G[:,i])

    qs[k:(k+i)] = r
    q2s[k:(k+i)] = r2

    k += i + 1

In [98]:
#%%timeit

rs = np.ones(N)
r2s = np.ones(N)

k = 0
for i in range(M):
    for j in range(i):
        a, b, c, d, r, r2 = cor_jackknife(G[:,(i,j)])
        rs[k] = r
        r2s[k] = r2
        k += 1
    k += 1

In [103]:
np.allclose(qs, rs)

True

In [104]:
np.allclose(q2s, r2s)

True